In [3]:
import requests
import json 
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm

urls = json.load(open("pandas_apis.json"))
data_frame_url = urls["df"]

def get_api_list(_url):
    page = requests.get(_url)
    soup = bs(page.content, 'html.parser')
    codes = soup.find_all("code", {"class": "xref py py-obj docutils literal notranslate"})
    apis = []
    for code in codes:
        api_name = code.find("span").text
        if "." in api_name:
            name = api_name[api_name.index(".") + 1:]
            apis.append(name)
            pass
        else:
            apis.append(api_name)
        pass
    return apis

data_frame_apis = get_api_list(data_frame_url)
# print(data_frame_apis)

list_of_apis = []
complete_api_sets = []

for key in urls:
    api_from_url = get_api_list(urls[key])
    list_of_apis.append({
        "url_key": key,
        "url": urls[key],
        "apis": api_from_url
    })
    complete_api_sets.extend(api_from_url)
    pass

In [25]:
full_apli_list = []
for apis in list_of_apis:
    full_apli_list.extend(apis['apis'])
    
full_apli_list = [api.strip() for api in full_apli_list]
print(full_apli_list)

['read_pickle', 'to_pickle', 'read_table', 'read_csv', 'to_csv', 'read_fwf', 'read_clipboard', 'to_clipboard', 'read_excel', 'to_excel', 'parse', 'to_excel', 'ExcelWriter', 'read_json', 'to_json', 'build_table_schema', 'read_html', 'to_html', 'to_html', 'read_xml', 'to_xml', 'to_latex', 'to_latex', 'read_hdf', 'put', 'append', 'get', 'select', 'info', 'keys', 'groups', 'walk', 'read_feather', 'to_feather', 'read_parquet', 'to_parquet', 'read_orc', 'read_sas', 'read_spss', 'read_sql_table', 'read_sql_query', 'read_sql', 'to_sql', 'read_gbq', 'read_stata', 'to_stata', 'data_label', 'value_labels', 'variable_labels', 'write_file', 'melt', 'pivot', 'pivot_table', 'crosstab', 'cut', 'qcut', 'merge', 'merge_ordered', 'merge_asof', 'concat', 'get_dummies', 'factorize', 'unique', 'wide_to_long', 'isna', 'isnull', 'notna', 'notnull', 'to_numeric', 'to_datetime', 'to_timedelta', 'date_range', 'bdate_range', 'period_range', 'timedelta_range', 'infer_freq', 'interval_range', 'eval', 'hash_array', 

In [7]:
import ast

In [8]:
import tokenize

In [9]:
from io import BytesIO
import tokenize
print(tokenize.ENCODING, tokenize.NL, tokenize.NEWLINE, tokenize.INDENT, tokenize.DEDENT, tokenize.ENDMARKER)

my_code = """
'''
adsfsdf
'''
def python_code_tokenize(full_code_text):
    g = tokenize.tokenize(BytesIO(full_code_text.encode('utf-8')).readline)
    tokens = []
    prev_token = None
    try:
        for x in g:
            if x.type == tokenize.ENDMARKER:  # End Marker
                continue
            # if x.type == tokenize.COMMENT:
            #     continue
            elif x.type == tokenize.NEWLINE:
                tokens.append('NEW_LINE')
            elif x.type == tokenize.INDENT:
                tokens.append("INDENT")
            elif x.type == tokenize.DEDENT:
                tokens.append('DEDENT')
            elif x.type == tokenize.STRING:  # String
                s = x.string.strip()
                if s.startswith('\"\"\"') or s.startswith("'''"):
                    if prev_token is not None and (prev_token == '=' or prev_token == '(' or prev_token == ','):
                        tokens.append(s)
                    continue
                tokens.append(s)
                pass
            elif x.type == tokenize.NL:
                continue
            elif x.type < 57:
                tokens.append(x.string)
            prev_token = x.string.strip()
    except:
        return []
        pass
return tokens
"""
def python_code_tokenize(full_code_text):
        '''
        :param full_code_text:
        :return:
        '''
        g = tokenize.tokenize(BytesIO(full_code_text.encode('utf-8')).readline)
        tokens = []
        prev_token = None
        for x in g:
            if x.type == tokenize.ENDMARKER:  # End Marker
                continue
            elif x.type == tokenize.NEWLINE:
                tokens.append('NEW_LINE')
            elif x.type == tokenize.INDENT:
                tokens.append('INDENT')
            elif x.type == tokenize.DEDENT:
                tokens.append('DEDENT')
            elif x.type == tokenize.STRING:  # String
                s = x.string.strip()
                if s.startswith('"""') or s.startswith("'''"):
                    if prev_token is not None and (prev_token == '=' or prev_token == '(' or prev_token == ','):
                        tokens.append(x.string)
                    continue
                tokens.append(x.string)
                pass
            elif x.string == '\n':
                continue
            elif x.type < 57:
                tokens.append(x.string)
            prev_token = x.string.strip()
        return tokens


def detokenize_python_code(tokens):
    indent_count = 0
    converted_tokens = []
    l = len(tokens)
    i = 0
    while i < l:
        t = tokens[i]
        if t == "NEW_LINE":
            converted_tokens.append("\n")
            for _ in range(indent_count):
                converted_tokens.append("  ")
        elif t == "INDENT":
            converted_tokens.append("  ")
            indent_count += 1
        elif t == "DEDENT":
            if converted_tokens[-1] == "  ":
                converted_tokens = converted_tokens[:-1]
            indent_count -= 1
        elif t == ".":
            if i+1 < l:
                token = converted_tokens[-1] + "." + tokens[i+1]
                converted_tokens = converted_tokens[:-1]
                converted_tokens.append(token)
                i += 1
            else:
                converted_tokens.append(".")
        else:
            converted_tokens.append(t)
        i += 1
    return " ".join(converted_tokens)

import keyword

def get_names(tokenized_code):
    tokens = tokenized_code.split()
    org_code = detokenize_python_code(tokens)
    names = []
    g = tokenize.tokenize(BytesIO(org_code.encode('utf-8')).readline)
    for x in g:
        if x.type == tokenize.NAME:
            st = x.string.strip()
            if st not in keyword.kwlist:
                names.append(st)
    return " <s> ".join(sorted(list(set(names))))

# get_names(" ".join(python_code_tokenize(my_code)))
# names = get_names(" ".join())

59 58 4 5 6 0


In [10]:
data_frame_api_re = "|".join(data_frame_apis)
print(data_frame_api_re)

DataFrame|index|columns|dtypes|info|select_dtypes|values|axes|ndim|size|shape|memory_usage|empty|set_flags|astype|convert_dtypes|infer_objects|copy|bool|head|at|iat|loc|iloc|insert|__iter__|items|iteritems|keys|iterrows|itertuples|lookup|pop|tail|xs|get|isin|where|mask|query|add|sub|mul|div|truediv|floordiv|mod|pow|dot|radd|rsub|rmul|rdiv|rtruediv|rfloordiv|rmod|rpow|lt|gt|le|ge|ne|eq|combine|combine_first|apply|applymap|pipe|agg|aggregate|transform|groupby|rolling|expanding|ewm|abs|all|any|clip|corr|corrwith|count|cov|cummax|cummin|cumprod|cumsum|describe|diff|eval|kurt|kurtosis|mad|max|mean|median|min|mode|pct_change|prod|product|quantile|rank|round|sem|skew|sum|std|var|nunique|value_counts|add_prefix|add_suffix|align|at_time|between_time|drop|drop_duplicates|duplicated|equals|filter|first|head|idxmax|idxmin|last|reindex|reindex_like|rename|rename_axis|reset_index|sample|set_axis|set_index|tail|take|truncate|backfill|bfill|dropna|ffill|fillna|interpolate|isna|isnull|notna|notnull|pad

In [43]:
import json 
examples = []
import re
import nltk

from tqdm.notebook import tqdm
fail_count = 0
example_id = 1
id_to_link = {}

for i in range(1, 4):
    example_file = f"25_K_Examples/part-{i}-output/taken_answers_with_all_details.json"
    data = json.load(open(example_file))
    for e in tqdm(data):
        try:
            qtitle = e['formatted_input']['question']['title']
            qdesc = e['formatted_input']['question']['ques_desc']
            io = " <s> ".join(e['formatted_input']['io'])
            codes = e['formatted_input']['answer']['code']
            apis = []
            for c in codes:
                tokens = nltk.wordpunct_tokenize(c)
                for token in tokens:
                    token = token.strip()
                    if token in full_apli_list:
                        apis.append(token)
            api_seq = " ".join(apis)
            
            question_apis = []
            q_tokens = nltk.wordpunct_tokenize(qtitle)
            q_tokens.extend(nltk.wordpunct_tokenize(qdesc))
            for q_t in q_tokens:
                q_t = q_t.strip()
                if q_t in full_apli_list:
                    question_apis.append(q_t)
            examples.append({
                'id': example_id,
                'q': qtitle.strip(),
                'd': qdesc.strip(),
                'q_apis': " ".join(question_apis),
                "io":  re.sub("[ \t\n]+", " ", io),
                "apis": api_seq,
                'code': codes,
                'link': e['link']
            })
            id_to_link[example_id] = e['link']
            example_id += 1
        except Exception as ex:
            print(ex)
            fail_count += 1
            
print(fail_count)
print(len(examples), len(id_to_link))

  0%|          | 0/206 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

  0%|          | 0/236 [00:00<?, ?it/s]

0
659 659


In [44]:
import numpy as np
def divide_ten_parts(indices, shuffle=True):
    if shuffle:
        np.random.shuffle(indices)
    partition_indices = []
    for i in range(10):
        ps = int(0.1 * i * len(indices))
        es = int(0.1 * (i + 1) * len(indices))
        train_indices = indices[0:ps]
        test_indices = indices[ps:es]
        train_indices.extend(indices[es:])
        valid_indices = train_indices[:int(0.1 * len(train_indices))]
        train_indices = train_indices[int(0.1 * len(train_indices)):]
        partition_indices.append((train_indices, valid_indices, test_indices))
    return partition_indices


partitions = divide_ten_parts(examples, shuffle=True)

In [45]:
import os 

os.makedirs("PLBART_EXPERIMENT/", exist_ok=True)

In [46]:
def write_examples(examples, d, part):
    _file = open(os.path.join(d, f"{part}.data"), 'w')
    for ex in examples:
        _file.write(json.dumps(ex) + "\n")
    _file.close()

for i, (tr, val, te) in enumerate(partitions):
    data_dir = f"PLBART_EXPERIMENT/DATA/part_{i}"
    os.makedirs(data_dir, exist_ok=True)
    write_examples(tr, data_dir, 'train')
    write_examples(val, data_dir, 'valid')
    write_examples(te, data_dir, 'test')
    
fp = open("PLBART_EXPERIMENT/DATA/id_to_link.json", 'w')
json.dump(id_to_link, fp, indent=4)
fp.close()

In [50]:
import argparse
import json
import os
from tqdm.notebook import tqdm

from multiprocessing import Pool
import sentencepiece as spm

import tokenize
from io import BytesIO


def python_code_tokenize(full_code_text):
    '''
    :param full_code_text:
    :return:
    '''
    g = tokenize.tokenize(BytesIO(full_code_text.encode('utf-8')).readline)
    tokens = []
    prev_token = None
    try:
        for x in g:
            if x.type == tokenize.ENDMARKER:  # End Marker
                continue
            # if x.type == tokenize.COMMENT:
            #     continue
            elif x.type == tokenize.NEWLINE:
                tokens.append('NEW_LINE')
            elif x.type == tokenize.INDENT:
                tokens.append('INDENT')
            elif x.type == tokenize.DEDENT:
                tokens.append('DEDENT')
            elif x.type == tokenize.STRING:  # String
                s = x.string.strip()
                if s.startswith('"""') or s.startswith("'''"):
                    if prev_token is not None and (prev_token == '=' or prev_token == '(' or prev_token == ','):
                        tokens.append(x.string)
                    continue
                tokens.append(x.string)
                pass
            elif x.string == '\n':
                continue
            elif x.type < 57:
                tokens.append(x.string)
            prev_token = x.string.strip()
    except:
        return []
        pass
    return tokens


class MultiprocessingEncoder(object):

    def __init__(self, model_file):
        self.model_file = model_file

    def initializer(self):
        global sp
        sp = spm.SentencePieceProcessor(model_file=self.model_file)

    def _encode(self, line):
        global sp
        return sp.encode(line, out_type=str)

    def _decode(self, tokens):
        global sp
        return sp.decode(tokens)

    def encode(self, example):
        assert isinstance(example, dict)
        assert 'src' in example and 'tgt' in example
        if len(example['src']) == 0 :
            return None
        if len(example['tgt']) == 0 :
            return None
        src_tokens = self._encode(example['src'])
        tgt_tokens = self._encode(example['tgt'])
        return {'src': " ".join(src_tokens), 'tgt': " ".join(tgt_tokens), 'link': example['link']}


def load_data(input_file, src_fields, tgt_field):
    data = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            ex = json.loads(line)
            assert tgt_field in ex
            srcs = []
            for sf in src_fields:
                assert sf in ex
                src = ex[sf]
                if isinstance(src, list):
                    src = " ".join(src)
                src = src.strip()
                if len(src) > 0:
                    srcs.append(src)
            src = " <s> ".join(srcs)
            tgt = ex[tgt_field]
            if isinstance(tgt, list):
                tgt = " ".join(tgt)
            tgt = tgt.replace('\n', ' ').strip()
            link = ex['link']
            data.append({'src': src, 'tgt': tgt, 'link': link})

    return data


def process(
    model_file, input_file, src_fields, tgt_field, 
    output_dir, part, pref, src_lang="en_XX", tgt_lang="py", workers=1
):
    dataset = load_data(input_file, src_fields, tgt_field)
    encoder = MultiprocessingEncoder(model_file)
    pool = Pool(workers, initializer=encoder.initializer)
    processed_dataset = []
    with tqdm(total=len(dataset), desc='Processing') as pbar:
        for i, ex in enumerate(pool.imap(encoder.encode, dataset, 100)):
            pbar.update()
            processed_dataset.append(ex)
    os.makedirs(output_dir, exist_ok=True)
    out_src = os.path.join(output_dir, '{}_{}.spm.{}'.format(pref, part, src_lang))
    out_tgt = os.path.join(output_dir, '{}_{}.spm.{}'.format(pref, part, tgt_lang))
    out_link = os.path.join(output_dir, '{}.link'.format(part))
    with open(out_src, 'w', encoding='utf-8') as src_writer, \
            open(out_tgt, 'w', encoding='utf-8') as tgt_writer, \
            open(out_link, 'w', encoding='utf-8') as link_writer:
        for ex in processed_dataset:
            if ex is not None:
                src_writer.write(ex['src'] + '\n')
                tgt_writer.write(ex['tgt'] + '\n')
                link_writer.write(ex['link'] + '\n')

                
# examples.append({
#                 'id': example_id,
#                 'q': qtitle.strip(),
#                 'd': qdesc.strip(),
#                 'q_apis': " ".join(question_apis),
#                 "io":  re.sub("[ \t\n]+", " ", io),
#                 "apis": api_seq,
#                 'code': codes,
#                 'link': e['link']
#             })
                
for i in range(10):
    data_dir = f"PLBART_EXPERIMENT/DATA/part_{i}"
    for part in ['train', 'valid', 'test']:
        process(
            input_file=os.path.join(data_dir, f"{part}.data"),
            src_fields=["q"],
            tgt_field="apis",
            output_dir=data_dir,
            pref=f"query",
            part=part,
            model_file="PLBART_EXPERIMENT/pretrained/sentencepiece.bpe.model"
        )
        
        process(
            input_file=os.path.join(data_dir, f"{part}.data"),
            src_fields=["q", "d"],
            tgt_field="apis",
            output_dir=data_dir,
            pref=f"query_desc",
            part=part,
            model_file="PLBART_EXPERIMENT/pretrained/sentencepiece.bpe.model"
        )
        
        process(
            input_file=os.path.join(data_dir, f"{part}.data"),
            src_fields=["q", "q_apis"],
            tgt_field="apis",
            output_dir=data_dir,
            pref=f"query_names",
            part=part,
            model_file="PLBART_EXPERIMENT/pretrained/sentencepiece.bpe.model"
        )
        
        process(
            input_file=os.path.join(data_dir, f"{part}.data"),
            src_fields=["q", "d", "io", "q_apis"],
            tgt_field="apis",
            output_dir=data_dir,
            pref=f"query_desc_io_names",
            part=part,
            model_file="PLBART_EXPERIMENT/pretrained/sentencepiece.bpe.model"
        )

Processing:   0%|          | 0/535 [00:00<?, ?it/s]

Processing:   0%|          | 0/535 [00:00<?, ?it/s]

Processing:   0%|          | 0/535 [00:00<?, ?it/s]

Processing:   0%|          | 0/535 [00:00<?, ?it/s]

Processing:   0%|          | 0/59 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/59 [00:00<?, ?it/s]

Processing:   0%|          | 0/66 [00:00<?, ?it/s]

Processing:   0%|          | 0/66 [00:00<?, ?it/s]

Processing:   0%|          | 0/66 [00:00<?, ?it/s]

Processing:   0%|          | 0/66 [00:00<?, ?it/s]